In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import numpy as np
import json
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Load data from 'hsg.json'
with open("hsg.json", "r") as file:
    data = json.load(file)

# Preparing dataset
features = np.array(
    [data["o"], data["h"], data["l"], data["c"], data["v"]]
).T  # Transpose to get a proper shape

# Normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_features = scaler.fit_transform(features)
scaled_features = scaled_features.astype(np.float32)


# Function to create sequences for LSTM
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        x = data[i : (i + seq_length)]
        y = data[i + seq_length, 1]  # Assuming 'h' (high prices) is at index 1
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)


seq_length = 5  # Number of time steps to look back
X, y = create_sequences(scaled_features, seq_length)

# Splitting the dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Define the RNN model
model = Sequential(
    [
        LSTM(
            50,
            activation="relu",
            return_sequences=True,
            input_shape=(seq_length, X.shape[2]),
        ),
        LSTM(50, activation="relu"),
        Dense(1),
    ]
)

# Compile the model
model.compile(optimizer="adam", loss="mse")

# Train the model
history = model.fit(
    X_train,
    y_train,
    epochs=50,
    validation_data=(X_test, y_test),
    batch_size=1024,
    verbose=1,
)

# Evaluate the model on the test set
test_loss = model.evaluate(X_test, y_test)

Epoch 1/50
3/3 [==============================] - 3s 602ms/step - loss: 0.0873 - val_loss: 0.0794
Epoch 2/50
3/3 [==============================] - 0s 81ms/step - loss: 0.0778 - val_loss: 0.0706
Epoch 3/50
3/3 [==============================] - 0s 80ms/step - loss: 0.0688 - val_loss: 0.0612
Epoch 4/50
3/3 [==============================] - 0s 91ms/step - loss: 0.0588 - val_loss: 0.0509
Epoch 5/50
3/3 [==============================] - 0s 80ms/step - loss: 0.0482 - val_loss: 0.0402
Epoch 6/50
3/3 [==============================] - 0s 82ms/step - loss: 0.0374 - val_loss: 0.0295
Epoch 7/50
3/3 [==============================] - 0s 85ms/step - loss: 0.0269 - val_loss: 0.0198
Epoch 8/50
3/3 [==============================] - 0s 80ms/step - loss: 0.0173 - val_loss: 0.0126
Epoch 9/50
3/3 [==============================] - 0s 90ms/step - loss: 0.0115 - val_loss: 0.0099
Epoch 10/50
3/3 [==============================] - 0s 85ms/step - loss: 0.0102 - val_loss: 0.0112
Epoch 11/50
3/3 [===========

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 5 and the array at index 1 has size 1

In [3]:
# Dự đoán giá mở cửa cho ngày tiếp theo
predicted_next_open_scaled = model.predict(
    scaled_features[-1].reshape(1, 1, scaled_features.shape[2])
)
predicted_next_open = custom_inverse_scale(predicted_next_open_scaled)

print(
    f"Giá mở cửa dự đoán cho ngày tiếp theo (đã đưa về quy mô ban đầu): {predicted_next_open[0][0]}"
)

1/1 [==============================] - 0s 34ms/step


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 5 and the array at index 1 has size 1